In [1]:
!pip install seaborn

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
Using cached seaborn-0.13.0-py3-none-any.whl (294 kB)


In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer

In [46]:
df = pd.read_csv("../data/data1.csv")

In [47]:
df.head()

,sentence
0,"Python is a high-level, general-purpose progra..."
1,Its design philosophy emphasizes code readabil...
2,"It supports multiple programming paradigms, in..."
3,"It is often described as a ""batteries included..."
4,Python 2.0 was released in 2000.


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4341 entries, 0 to 4340
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  4341 non-null   object
dtypes: object(1)
memory usage: 34.0+ KB


In [49]:
df.shape

(4341, 1)

In [50]:
df.isnull().sum()

sentence    0
dtype: int64

In [51]:
df.duplicated().sum()

0

In [52]:
sentences = df['sentence'].head(100).tolist()

In [53]:
tokenizer = Tokenizer()
def textTokenizer(text):
    token = tokenizer.fit_on_texts(text)

In [54]:
textTokenizer(sentences)

In [55]:
len(sentences[0])

61

In [56]:
#tokenizer.word_index

In [57]:
input_sequences = []

for sentence in sentences:
    tokenized_sentences = tokenizer.texts_to_sequences([sentence])[0]

    for i in range(1, len(tokenized_sentences)):
        n_gram = tokenized_sentences[:i+1]
        input_sequences.append(n_gram)

In [58]:
# input_sequences

In [16]:
max_len = max([len(x) for x in input_sequences])

In [17]:
max_len

77

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [19]:
total_words = len(tokenizer.word_index) + 1
total_words

935

In [20]:
X = padded_input_sequences[:,:-1]

In [21]:
y = padded_input_sequences[:,-1]

In [22]:
len(X[1])

76

In [23]:
X.shape

(2221, 76)

In [24]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_words)

In [25]:
y.shape

(2221, 935)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
X_train.shape

(1488, 76)

In [28]:
y_train.shape

(1488, 935)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Bidirectional
from sklearn.metrics import precision_score, recall_score, f1_score

In [30]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_len-1))
model.add(Bidirectional(GRU(150)))
model.add(Dense(total_words, activation='softmax'))

2023-12-26 11:33:05.995231: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-26 11:33:05.997149: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-26 11:33:05.999105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 76, 100)           93500     
                                                                 
 bidirectional (Bidirectiona  (None, 300)              226800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 935)               281435    
                                                                 
Total params: 601,735
Trainable params: 601,735
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100


2023-12-26 11:33:06.619296: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-26 11:33:06.621329: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-26 11:33:06.623306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

47/47 [==============================] - ETA: 0s - loss: 6.5975 - accuracy: 0.0363

2023-12-26 11:33:18.145751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-26 11:33:18.147974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-26 11:33:18.149684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

47/47 [==============================] - 13s 195ms/step - loss: 6.5975 - accuracy: 0.0363 - val_loss: 6.3183 - val_accuracy: 0.0532
Epoch 2/100
47/47 [==============================] - 7s 157ms/step - loss: 6.0449 - accuracy: 0.0397 - val_loss: 6.5150 - val_accuracy: 0.0532
Epoch 3/100
47/47 [==============================] - 8s 162ms/step - loss: 5.8956 - accuracy: 0.0491 - val_loss: 6.5833 - val_accuracy: 0.0641
Epoch 4/100
47/47 [==============================] - 7s 156ms/step - loss: 5.6888 - accuracy: 0.0578 - val_loss: 6.7243 - val_accuracy: 0.0859
Epoch 5/100
47/47 [==============================] - 9s 183ms/step - loss: 5.3505 - accuracy: 0.0948 - val_loss: 6.9761 - val_accuracy: 0.0955
Epoch 6/100
47/47 [==============================] - 7s 159ms/step - loss: 4.9122 - accuracy: 0.1290 - val_loss: 7.0364 - val_accuracy: 0.0805
Epoch 7/100
47/47 [==============================] - 8s 173ms/step - loss: 4.4179 - accuracy: 0.1707 - val_loss: 7.1368 - val_accuracy: 0.0832
Epoch 8/10

In [44]:
text = "Nothing"

for i in range(9):
  token_text = tokenizer.texts_to_sequences([text])[0]
  padded_token_text = pad_sequences([token_text], maxlen=max_len-1, padding='pre')
  result = model.predict(padded_token_text)

  pos = np.argmax(result)

  for word, index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)

1/1 [==============================] - 0s 64ms/step
Nothing intelligence
1/1 [==============================] - 0s 39ms/step
Nothing intelligence ai
1/1 [==============================] - 0s 35ms/step
Nothing intelligence ai is
1/1 [==============================] - 0s 47ms/step
Nothing intelligence ai is central
1/1 [==============================] - 0s 47ms/step
Nothing intelligence ai is central to
1/1 [==============================] - 0s 53ms/step
Nothing intelligence ai is central to programming
1/1 [==============================] - 0s 39ms/step
Nothing intelligence ai is central to programming languages
1/1 [==============================] - 0s 30ms/step
Nothing intelligence ai is central to programming languages derived
1/1 [==============================] - 0s 43ms/step
Nothing intelligence ai is central to programming languages derived from
